# SHAP Values from Scratch

## Understanding Shapley Additive Explanations

This notebook implements SHAP (SHapley Additive exPlanations) values from first principles to help you understand how the algorithm works.

### What are SHAP Values?

SHAP values explain individual predictions by computing the contribution of each feature to the prediction. They answer the question: **"How much did each feature push the prediction away from the baseline?"**

SHAP is based on **Shapley values** from cooperative game theory, which provide a fair way to distribute credit among team members (or in our case, features).

### Key Concepts

1. **Coalition**: A subset of features being considered
2. **Marginal Contribution**: The change in prediction when adding a feature to a coalition
3. **Shapley Value**: Weighted average of marginal contributions across all possible coalitions
4. **Baseline**: A reference prediction (often using mean/median feature values)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
from typing import List
from math import factorial
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")

## Part 1: Understanding Shapley Values with a Simple Example

Let's start with a simple linear model to build intuition:

**Model**: `prediction = 2×x₁ + 3×x₂ + 1×x₃`

We'll calculate how much each feature contributes to the final prediction.

In [ ]:
def simple_prediction_function(x1, x2, x3):
    """
    Simple linear model for demonstration.
    
    prediction = 2*x1 + 3*x2 + 1*x3
    
    This represents our "black box" model that we want to explain.
    """
    return 2*x1 + 3*x2 + 1*x3

# Example instance to explain
instance = {'x1': 5, 'x2': 3, 'x3': 2}
baseline = {'x1': 0, 'x2': 0, 'x3': 0}  # All features at zero

# Calculate predictions
prediction = simple_prediction_function(**instance)
baseline_pred = simple_prediction_function(**baseline)

print("=" * 60)
print("EXAMPLE INSTANCE")
print("=" * 60)
print(f"Instance values: {instance}")
print(f"Prediction: {prediction}")
print(f"Baseline prediction: {baseline_pred}")
print(f"\nDifference to explain: {prediction - baseline_pred}")
print("\nGoal: Distribute this difference fairly among features")

## Part 2: The Core SHAP Algorithm

### Algorithm Overview

To calculate the SHAP value for a feature, we:

1. **Enumerate all coalitions** (subsets) that DON'T include the feature
2. **For each coalition S**:
   - Calculate prediction with just coalition S: `v(S)`
   - Calculate prediction with S plus our feature: `v(S ∪ {feature})`
   - Compute **marginal contribution**: `v(S ∪ {feature}) - v(S)`
   - Weight it by coalition probability
3. **Sum all weighted contributions**

### The Weight Formula

For a coalition of size |S| with n total features:

**Weight = |S|! × (n - |S| - 1)! / n!**

This ensures fair credit distribution (from game theory).

In [ ]:
def get_all_coalitions(features: List[str]) -> List[tuple]:
    """
    Generate all possible coalitions (subsets) of features.
    
    Example: For [A, B, C], generates:
    [], [A], [B], [C], [A,B], [A,C], [B,C], [A,B,C]
    """
    coalitions = []
    for size in range(len(features) + 1):
        for coalition in combinations(features, size):
            coalitions.append(coalition)
    return coalitions

# Test it
test_features = ['A', 'B', 'C']
all_coalitions = get_all_coalitions(test_features)
print(f"All coalitions for {test_features}:")
for i, coalition in enumerate(all_coalitions):
    print(f"  {i+1}. {list(coalition) if coalition else 'empty'}")
print(f"\nTotal coalitions: {len(all_coalitions)} (= 2^{len(test_features)})") 

In [ ]:
def predict_with_coalition(model_func, instance, coalition, baseline, all_features):
    """
    Make prediction using only features in the coalition.
    
    - Features IN coalition: use instance values
    - Features NOT in coalition: use baseline values (marginalize)
    
    This simulates: "What if we only knew these features?"
    """
    values = {}
    for feature in all_features:
        if feature in coalition:
            values[feature] = instance[feature]
        else:
            values[feature] = baseline[feature]
    return model_func(**values)

# Example: predict with only x1 and x3
coalition_example = ['x1', 'x3']
pred_example = predict_with_coalition(
    simple_prediction_function, 
    instance, 
    coalition_example, 
    baseline, 
    ['x1', 'x2', 'x3']
)

print(f"Coalition: {coalition_example}")
print(f"Using: x1={instance['x1']}, x2={baseline['x2']} (baseline), x3={instance['x3']}")
print(f"Prediction: {pred_example}")
print(f"Calculation: 2×{instance['x1']} + 3×{baseline['x2']} + 1×{instance['x3']} = {pred_example}")

In [ ]:
def calculate_shapley_value(feature: str, model_func, instance, baseline, all_features):
    """
    Calculate exact Shapley value for a single feature.
    
    Returns:
        shapley_value: The SHAP value for this feature
        contributions: List of detailed marginal contributions (for debugging)
    """
    n = len(all_features)
    other_features = [f for f in all_features if f != feature]
    
    shapley_value = 0.0
    contributions = []
    
    # Consider all coalitions that DON'T include our feature
    for coalition_size in range(n):
        for coalition in combinations(other_features, coalition_size):
            coalition = list(coalition)
            
            # Prediction WITHOUT the feature
            pred_without = predict_with_coalition(
                model_func, instance, coalition, baseline, all_features
            )
            
            # Prediction WITH the feature added
            coalition_with_feature = coalition + [feature]
            pred_with = predict_with_coalition(
                model_func, instance, coalition_with_feature, baseline, all_features
            )
            
            # Marginal contribution of adding this feature
            marginal_contribution = pred_with - pred_without
            
            # Calculate Shapley weight
            coalition_size = len(coalition)
            weight = (
                factorial(coalition_size) * 
                factorial(n - coalition_size - 1) / 
                factorial(n)
            )
            
            weighted_contribution = weight * marginal_contribution
            shapley_value += weighted_contribution
            
            contributions.append({
                'coalition': coalition,
                'pred_without': pred_without,
                'pred_with': pred_with,
                'marginal': marginal_contribution,
                'weight': weight,
                'weighted': weighted_contribution
            })
    
    return shapley_value, contributions

print("✓ Shapley value calculation function defined")

## Part 3: Detailed Calculation for One Feature

Let's trace through the calculation for `x1` step by step to see how SHAP values are computed.

In [ ]:
# Calculate SHAP value for x1
all_features = ['x1', 'x2', 'x3']
shap_x1, contributions_x1 = calculate_shapley_value(
    'x1', 
    simple_prediction_function, 
    instance, 
    baseline, 
    all_features
)

print("=" * 70)
print("DETAILED CALCULATION FOR x1")
print("=" * 70)
print("\nAll marginal contributions:\n")

# Show all contributions
for i, contrib in enumerate(contributions_x1):
    coalition_str = str(contrib['coalition']) if contrib['coalition'] else 'empty'
    print(f"Coalition {i+1}: {coalition_str}")
    print(f"  Pred without x1: {contrib['pred_without']:6.2f}")
    print(f"  Pred with x1:    {contrib['pred_with']:6.2f}")
    print(f"  Marginal:        {contrib['marginal']:6.2f}")
    print(f"  Weight:          {contrib['weight']:6.4f}")
    print(f"  Weighted:        {contrib['weighted']:6.2f}")
    print()

print(f"FINAL SHAP VALUE for x1: {shap_x1:.2f}")

## Part 4: Calculate SHAP Values for All Features

Now let's calculate SHAP values for all features and verify the key property: **SHAP values sum to (prediction - baseline)**.

In [ ]:
# Calculate for all features
shapley_values = {}

for feature in all_features:
    shap_val, _ = calculate_shapley_value(
        feature, 
        simple_prediction_function, 
        instance, 
        baseline, 
        all_features
    )
    shapley_values[feature] = shap_val

print("=" * 70)
print("FINAL SHAP VALUES")
print("=" * 70)
print("\nFeature contributions:")
for feat, val in shapley_values.items():
    print(f"  {feat}: {val:7.2f}")

print(f"\nSum of SHAP values:        {sum(shapley_values.values()):.2f}")
print(f"Prediction - Baseline:     {prediction - baseline_pred:.2f}")
print(f"\n{'✓' * 35}")
print("VERIFICATION: SHAP values sum to (prediction - baseline)!")
print(f"{'✓' * 35}")

## Part 5: Visualization

A waterfall or bar plot shows how each feature contributes to moving the prediction from baseline to final value.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

features = list(shapley_values.keys())
values = list(shapley_values.values())
colors = ['#2ecc71' if v > 0 else '#e74c3c' for v in values]

# Create horizontal bar chart
bars = ax.barh(features, values, color=colors, alpha=0.7, edgecolor='black')

# Add zero line
ax.axvline(x=0, color='black', linestyle='-', linewidth=1)

# Labels and title
ax.set_xlabel('SHAP Value (contribution to prediction)', fontsize=12, fontweight='bold')
ax.set_ylabel('Features', fontsize=12, fontweight='bold')
ax.set_title('SHAP Values: Feature Contributions to Prediction', 
             fontsize=14, fontweight='bold', pad=20)
ax.grid(axis='x', alpha=0.3, linestyle='--')

# Add value labels on bars
for i, (feat, val) in enumerate(zip(features, values)):
    offset = 0.3 if val > 0 else -0.3
    ha = 'left' if val > 0 else 'right'
    ax.text(val + offset, i, f'{val:.2f}', 
            va='center', ha=ha, fontsize=11, fontweight='bold')

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#2ecc71', alpha=0.7, label='Positive (increases prediction)'),
    Patch(facecolor='#e74c3c', alpha=0.7, label='Negative (decreases prediction)')
]
ax.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("  • Positive (green): Feature pushes prediction UP")
print("  • Negative (red): Feature pushes prediction DOWN")
print("  • Magnitude: How strongly the feature influences prediction")

## Part 6: Real Machine Learning Example

Now let's apply SHAP to a real ML model (Decision Tree) trained on synthetic data.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import make_regression

# Create synthetic dataset
print("Creating dataset and training model...")
X, y = make_regression(n_samples=200, n_features=4, noise=15, random_state=42)
feature_names = ['Feature_A', 'Feature_B', 'Feature_C', 'Feature_D']
X_df = pd.DataFrame(X, columns=feature_names)

# Train decision tree
model = DecisionTreeRegressor(max_depth=4, random_state=42)
model.fit(X, y)

print(f"✓ Model trained (R² score: {model.score(X, y):.3f})")
print(f"\nDataset shape: {X_df.shape}")
print(f"\nFirst few rows:")
print(X_df.head())

In [ ]:
# Select instance to explain
instance_idx = 5
instance_to_explain = X_df.iloc[instance_idx].to_dict()

# Use mean as baseline
baseline_values = {feat: X_df[feat].mean() for feat in feature_names}

print("=" * 70)
print(f"INSTANCE TO EXPLAIN (index {instance_idx})")
print("=" * 70)
print("\nFeature values:")
for feat, val in instance_to_explain.items():
    baseline_val = baseline_values[feat]
    diff = val - baseline_val
    print(f"  {feat:12} = {val:7.3f}  (baseline: {baseline_val:6.3f}, diff: {diff:+7.3f})")

# Model wrapper
def model_predict(**kwargs):
    """Wrapper to make sklearn model compatible with our SHAP calculator"""
    feature_values = [kwargs[feat] for feat in feature_names]
    return model.predict([feature_values])[0]

instance_pred = model_predict(**instance_to_explain)
baseline_pred = model_predict(**baseline_values)

print(f"\nPredictions:")
print(f"  Instance:  {instance_pred:7.3f}")
print(f"  Baseline:  {baseline_pred:7.3f}")
print(f"  Difference: {instance_pred - baseline_pred:+7.3f}")

In [ ]:
# Calculate SHAP values for ML model
print("Calculating SHAP values...")
print("(This may take 10-20 seconds due to 2^4 = 16 coalitions per feature)\n")

ml_shapley_values = {}

for i, feature in enumerate(feature_names, 1):
    shap_val, _ = calculate_shapley_value(
        feature, 
        model_predict, 
        instance_to_explain, 
        baseline_values, 
        feature_names
    )
    ml_shapley_values[feature] = shap_val
    print(f"[{i}/4] {feature:12} SHAP value: {shap_val:+7.3f}")

print("\n" + "=" * 70)
print("VERIFICATION")
print("=" * 70)
print(f"Sum of SHAP values:    {sum(ml_shapley_values.values()):7.3f}")
print(f"Prediction - Baseline: {instance_pred - baseline_pred:7.3f}")
print(f"Difference:            {abs(sum(ml_shapley_values.values()) - (instance_pred - baseline_pred)):.6f}")
print("\n✓ Local accuracy property satisfied!")

In [ ]:
# Visualize ML model SHAP values
fig, ax = plt.subplots(figsize=(10, 6))

ml_features = list(ml_shapley_values.keys())
ml_values = list(ml_shapley_values.values())
ml_colors = ['#2ecc71' if v > 0 else '#e74c3c' for v in ml_values]

# Sort by absolute value
sorted_indices = np.argsort([abs(v) for v in ml_values])
ml_features_sorted = [ml_features[i] for i in sorted_indices]
ml_values_sorted = [ml_values[i] for i in sorted_indices]
ml_colors_sorted = [ml_colors[i] for i in sorted_indices]

bars = ax.barh(ml_features_sorted, ml_values_sorted, 
               color=ml_colors_sorted, alpha=0.7, edgecolor='black')

ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
ax.set_xlabel('SHAP Value (contribution to prediction)', fontsize=12, fontweight='bold')
ax.set_ylabel('Features', fontsize=12, fontweight='bold')
ax.set_title(f'SHAP Values for Decision Tree (Instance {instance_idx})', 
             fontsize=14, fontweight='bold', pad=20)
ax.grid(axis='x', alpha=0.3, linestyle='--')

# Add value labels
for i, (feat, val) in enumerate(zip(ml_features_sorted, ml_values_sorted)):
    offset = 0.5 if val > 0 else -0.5
    ha = 'left' if val > 0 else 'right'
    ax.text(val + offset, i, f'{val:.2f}', 
            va='center', ha=ha, fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

# Feature importance ranking
print("\nFeature Importance Ranking (by absolute SHAP value):")
importance_rank = sorted(ml_shapley_values.items(), 
                         key=lambda x: abs(x[1]), reverse=True)
for rank, (feat, val) in enumerate(importance_rank, 1):
    print(f"  {rank}. {feat:12} = {val:+7.3f}")

## Part 7: Understanding Computational Complexity

The exact SHAP calculation requires evaluating all possible coalitions.

**Complexity**: O(2^n) where n = number of features

| Features | Coalitions | Approximate Time |
|----------|-----------|------------------|
| 4        | 16        | < 1 second      |
| 10       | 1,024     | ~1 second       |
| 15       | 32,768    | ~30 seconds     |
| 20       | 1,048,576 | ~15 minutes     |
| 30       | 1B+       | Infeasible      |

This is why practical implementations use approximations:
- **TreeSHAP**: Polynomial time for tree-based models
- **KernelSHAP**: Sampling-based approximation
- **Linear SHAP**: Closed-form for linear models

In [ ]:
# Demonstrate complexity
def count_coalitions(n_features):
    return 2 ** n_features

print("Number of coalitions to evaluate:\n")
for n in [4, 6, 8, 10, 12, 15, 20]:
    coalitions = count_coalitions(n)
    print(f"  {n:2} features: {coalitions:>10,} coalitions")
    
print("\nFor the simple example (3 features):")
print(f"  Total coalitions: {count_coalitions(3)}")
print(f"  Coalitions per feature: {count_coalitions(2)} (excluding the feature itself)")
print(f"  Total evaluations for all features: {3 * count_coalitions(2)} model calls")

## Part 8: Sampling-Based Approximation (Optional)

For large feature sets, we can approximate SHAP values by sampling random permutations instead of enumerating all coalitions.

**Permutation Algorithm**:
1. Randomly order all features
2. For each position, compute marginal contribution when our feature appears
3. Average across many random orderings

In [ ]:
def calculate_shap_permutation(feature: str, model_func, instance, 
                                baseline, all_features, n_samples: int = 200):
    """
    Approximate SHAP using random permutations (faster for many features).
    
    This is similar to the approach used in KernelSHAP.
    """
    n = len(all_features)
    other_features = [f for f in all_features if f != feature]
    
    contributions = []
    
    for _ in range(n_samples):
        # Random permutation of other features
        perm = list(np.random.permutation(other_features))
        
        # Try each insertion position
        for insert_pos in range(len(perm) + 1):
            coalition_before = perm[:insert_pos]
            
            pred_without = predict_with_coalition(
                model_func, instance, coalition_before, baseline, all_features
            )
            
            coalition_with = coalition_before + [feature]
            pred_with = predict_with_coalition(
                model_func, instance, coalition_with, baseline, all_features
            )
            
            contributions.append(pred_with - pred_without)
    
    return np.mean(contributions)

# Compare exact vs approximate
print("Comparing Exact vs. Approximate SHAP:\n")
print("Feature      Exact    Approx   Difference")
print("-" * 50)

for feature in all_features:
    exact = shapley_values[feature]
    approx = calculate_shap_permutation(
        feature, simple_prediction_function, instance, 
        baseline, all_features, n_samples=500
    )
    diff = abs(exact - approx)
    print(f"{feature:8}  {exact:7.3f}  {approx:7.3f}  {diff:7.4f}")

print("\n✓ Approximation is very close to exact values!")

## Part 9: SHAP Properties and Theory

SHAP values satisfy three important axioms from cooperative game theory:

### 1. Local Accuracy (Efficiency)
**Sum of SHAP values = Prediction - Baseline**

The SHAP values completely explain the difference between the prediction and baseline.

### 2. Missingness
**If a feature is absent (at baseline), its SHAP value is 0**

Features that don't differ from baseline don't contribute.

### 3. Consistency
**If a model changes so a feature has larger marginal contributions, its SHAP value doesn't decrease**

SHAP values respect the feature's actual importance.

### Comparison with Other Methods

| Method | Pros | Cons |
|--------|------|------|
| **SHAP** | Theoretically grounded, local explanations, additive | Computationally expensive |
| **LIME** | Fast, model-agnostic | No theoretical guarantees, unstable |
| **Feature Importance** | Fast, global view | Not instance-specific |
| **Partial Dependence** | Shows average effect | Not for individual predictions |

In [ ]:
# Verify local accuracy property
print("=" * 70)
print("VERIFICATION OF LOCAL ACCURACY PROPERTY")
print("=" * 70)

print("\n1. Simple Linear Model:")
sum_shap_simple = sum(shapley_values.values())
diff_simple = prediction - baseline_pred
print(f"   Sum of SHAP values:    {sum_shap_simple:.6f}")
print(f"   Prediction - Baseline: {diff_simple:.6f}")
print(f"   Difference:            {abs(sum_shap_simple - diff_simple):.10f}")

print("\n2. ML Model (Decision Tree):")
sum_shap_ml = sum(ml_shapley_values.values())
diff_ml = instance_pred - baseline_pred
print(f"   Sum of SHAP values:    {sum_shap_ml:.6f}")
print(f"   Prediction - Baseline: {diff_ml:.6f}")
print(f"   Difference:            {abs(sum_shap_ml - diff_ml):.10f}")

print("\n✓ Both models satisfy local accuracy!")
print("  (Small numerical differences are due to floating-point precision)")

## Summary and Key Takeaways

### What We Learned

1. **SHAP Algorithm**: 
   - Enumerate all coalitions (feature subsets)
   - Compute marginal contribution of each feature
   - Weight by coalition probability
   - Sum to get final SHAP value

2. **Interpretation**:
   - Positive SHAP → feature increases prediction
   - Negative SHAP → feature decreases prediction
   - Magnitude → strength of influence

3. **Properties**:
   - Local accuracy: values sum to (prediction - baseline)
   - Theoretically grounded in game theory
   - Model-agnostic: works with any ML model

4. **Computational Cost**:
   - Exact: O(2^n) - exponential in features
   - Approximations needed for >15 features
   - TreeSHAP, KernelSHAP provide efficient alternatives

### When to Use SHAP

✅ **Good for**:
- Explaining individual predictions
- Understanding feature importance locally
- Debugging model behavior
- Building trust in model decisions

❌ **Limitations**:
- Computationally expensive for many features
- Requires choosing appropriate baseline
- Doesn't capture feature interactions explicitly

### Next Steps

1. Try **TreeSHAP** for tree-based models (XGBoost, Random Forest)
2. Use **KernelSHAP** for general models with many features
3. Compare SHAP with LIME for your use case
4. Explore SHAP interaction values for feature pairs

### References

- Original paper: [A Unified Approach to Interpreting Model Predictions](https://arxiv.org/abs/1705.07874)
- SHAP library: https://github.com/slundberg/shap
- Game theory background: [Shapley Value](https://en.wikipedia.org/wiki/Shapley_value)